In [1]:
from depiction.models.celltype import CellTyper

In [2]:
classifier = CellTyper()

114688/108296 [===============================] - 0s 1us/step


W0703 18:32:51.250677 4626900416 deprecation.py:506] From /Users/dow/anaconda3/envs/depiction-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0703 18:32:51.251670 4626900416 deprecation.py:506] From /Users/dow/anaconda3/envs/depiction-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0703 18:32:51.254011 4626900416 deprecation.py:506] From /Users/dow/anaconda3/envs/depiction-env/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97